In [0]:
spark.conf.set("fs.azure.account.key.dpstorage21.dfs.core.windows.net","o1zhSIwsiONj8+NSuytF22pskSpGLY4gKUIYWTkpNoGUOJrmfVGdIvdzDYByuwtBmtMtyvm0OB39+AStJwZKlQ==")

In [0]:
#service_credential = "p-J8Q~C6Fc8tFwtS-QAE-qFP9EPuXE4SV5McAc5Q"
#application_id = "552a69ea-ab44-4842-8a03-7bcc62f38926"
#directory_id = "b99772b5-42c9-4cf6-829d-232ec2fbe8e7"

In [0]:

#configs = {"fs.azure.account.auth.type": "OAuth",
         # "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
         # "fs.azure.account.oauth2.client.id": application_id,
         # "fs.azure.account.oauth2.client.secret":service_credential,
         #"fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{directory_id}/oauth2/token"}

In [0]:
display(dbutils.fs.ls("abfss://raw@dpstorage21.dfs.core.windows.net"))

path,name,size,modificationTime
abfss://raw@dpstorage21.dfs.core.windows.net/sales_data.csv,sales_data.csv,527958,1733365140000


Extract

In [0]:
df = spark.read.csv("abfss://raw@dpstorage21.dfs.core.windows.net/sales_data.csv", header=True, inferSchema="true");
df.show(25)

+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+--------+----------+---------+---------+---------------+----------------+--------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|  SALES|      ORDERDATE| STATUS|QTR_ID|MONTH_ID|YEAR_ID|PRODUCTLINE|MSRP|PRODUCTCODE|        CUSTOMERNAME|           PHONE|        ADDRESSLINE1|ADDRESSLINE2|         CITY|   STATE|POSTALCODE|  COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|
+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+--------+----------+---------+---------+---------------+----------------+--------+
|      10107|             30|     95.7|              2| 2871.0| 2/24/2003

In [0]:
%python
# Install the required package
%pip install fsspec

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
 %restart_python

In [0]:
%python
from pyspark.sql.functions import col
import pandas as pd

# Load data using Spark
file_path = 'abfss://raw@dpstorage21.dfs.core.windows.net/sales_data.csv'  # Update for Databricks FileStore
sales_data_spark = spark.read.csv(file_path, header=True, inferSchema=True, encoding='latin1')

# Convert Spark DataFrame to Pandas DataFrame
sales_data = sales_data_spark.toPandas()

# Print column names to verify
print(sales_data.columns)

# 1. Handle Missing Values
# Drop columns with >50% missing values
threshold = len(sales_data) * 0.5
sales_data_cleaned = sales_data.dropna(axis=1, thresh=threshold)

# Impute missing values in 'STATE' with 'Unknown'
if 'STATE' in sales_data_cleaned.columns:
    sales_data_cleaned['STATE'] = sales_data_cleaned['STATE'].fillna('Unknown')
else:
    print("Column 'STATE' not found in the DataFrame")

# Impute missing values in 'POSTALCODE' with 0
if 'POSTALCODE' in sales_data_cleaned.columns:
    sales_data_cleaned['POSTALCODE'] = sales_data_cleaned['POSTALCODE'].fillna(0)
else:
    print("Column 'POSTALCODE' not found in the DataFrame")

# 2. Convert Data Types
# Convert 'ORDERDATE' to datetime format
if 'ORDERDATE' in sales_data_cleaned.columns:
    sales_data_cleaned['ORDERDATE'] = pd.to_datetime(sales_data_cleaned['ORDERDATE'])
else:
    print("Column 'ORDERDATE' not found in the DataFrame")

# 3. Remove Duplicates
sales_data_cleaned = sales_data_cleaned.drop_duplicates()

# 4. Filter Invalid or Outlier Data
# Remove negative or zero values in 'SALES'
if 'SALES' in sales_data_cleaned.columns:
    sales_data_cleaned = sales_data_cleaned[sales_data_cleaned['SALES'] > 0]
else:
    print("Column 'SALES' not found in the DataFrame")

# 5. Add New Features
# Create a 'PROFIT' column: Assume a 20% profit margin
if 'SALES' in sales_data_cleaned.columns:
    sales_data_cleaned['PROFIT'] = sales_data_cleaned['SALES'] * 0.2
else:
    print("Column 'SALES' not found in the DataFrame")

# Convert cleaned Pandas DataFrame back to Spark DataFrame
sales_data_cleaned_spark = spark.createDataFrame(sales_data_cleaned)

# Save cleaned data
output_path = 'abfss://processed@dpstorage21.dfs.core.windows.net/sales_data_cleaned.csv'  # Update for Databricks FileStore
sales_data_cleaned_spark.write.csv(output_path, header=True, mode='overwrite')

print("Data cleaning complete. Cleaned dataset saved.")

Index(['ORDERNUMBER', 'QUANTITYORDERED', 'PRICEEACH', 'ORDERLINENUMBER',
       'SALES', 'ORDERDATE', 'STATUS', 'QTR_ID', 'MONTH_ID', 'YEAR_ID',
       'PRODUCTLINE', 'MSRP', 'PRODUCTCODE', 'CUSTOMERNAME', 'PHONE',
       'ADDRESSLINE1', 'ADDRESSLINE2', 'CITY', 'STATE', 'POSTALCODE',
       'COUNTRY', 'TERRITORY', 'CONTACTLASTNAME', 'CONTACTFIRSTNAME',
       'DEALSIZE'],
      dtype='object')
Column 'STATE' not found in the DataFrame


/root/.ipykernel/13417/command-583973560225802-3323106881:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_data_cleaned['POSTALCODE'] = sales_data_cleaned['POSTALCODE'].fillna(0)
/root/.ipykernel/13417/command-583973560225802-3323106881:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_data_cleaned['ORDERDATE'] = pd.to_datetime(sales_data_cleaned['ORDERDATE'])
/databricks/spark/python/pyspark/sql/pandas/conversion.py:477: UserWarning: createDataFrame attempted Arrow optimization because 'spark.

Data cleaning complete. Cleaned dataset saved.
